In [3]:
import os
import json
import numpy as np

In [5]:
def f1_from_nonzero(A_true, A_est):
    A_true_nz = (A_true != 0)
    A_est_nz  = (A_est  != 0)

    TP = np.sum(A_true_nz & A_est_nz)
    FP = np.sum(~A_true_nz & A_est_nz)
    FN = np.sum(A_true_nz & ~A_est_nz)

    precision = TP / (TP + FP + 1e-12)
    recall    = TP / (TP + FN + 1e-12)
    f1        = 2 * precision * recall / (precision + recall + 1e-12)

    return TP, FP, FN, precision, recall, f1

In [20]:
folder = "result_d4_s3(new_reg)"  

W1_thres = 0.3
W2_thres = 0.2

# Track best files
best_f1_W1 = -1.0
best_f1_W1_file = None

best_f1_W2 = -1.0
best_f1_W2_file = None

best_avg_f1 = -1.0
best_avg_f1_file = None

for filename in os.listdir(folder):

    file_path = os.path.join(folder, filename)
    print("Processing:", file_path)

    with open(file_path, "r") as f:
        data = json.load(f)

    # --- truth ---
    W_truth = np.array(data["W_truth"])
    Sigma_truth = np.array(data["Sigma_truth"])  # assumes this key exists

    W_truth[np.abs(W_truth) < W1_thres] = 0
    Sigma_truth[np.abs(Sigma_truth) < W2_thres] = 0

    # --- pick run_id == 0 ---
    W_est = None
    Sigma_est = None
    chosen_run = None

    for run in data["random_runs"]:
        if run["run_id"] == 0:
            chosen_run = run
            W_est = np.array(run["W_est_random"])
            Sigma_est = np.array(run["Sigma_est_random"])
            break

    if W_est is None or Sigma_est is None:
        print(f"  Skipping {filename}: run_id == 0 not found.")
        continue

    # --- threshold estimates ---
    W_est[np.abs(W_est) < W1_thres] = 0
    Sigma_est[np.abs(Sigma_est) < W2_thres] = 0

    # --- F1 for W ---
    TP_W1, FP_W1, FN_W1, precision_W1, recall_W1, f1_W1 = f1_from_nonzero(W_truth, W_est)

    # --- F1 for Sigma (strictly lower-triangular) ---
    TP_W2, FP_W2, FN_W2, precision_W2, recall_W2, f1_W2 = f1_from_nonzero(
        np.tril(Sigma_truth, k=-1),
        np.tril(Sigma_est,   k=-1),
    )

    # --- save metrics into JSON ---
    data["f1_W1"] = round(f1_W1, 4)
    data["f1_W2"] = round(f1_W2, 4)
    # print("TP_W1", TP_W1)
    data["TP_W1"] = TP_W1.item()
    data["FP_W1"] = FP_W1.item()
    data["FN_W1"] = FN_W1.item()

    data["TP_W2"] = TP_W2.item()
    data["FP_W2"] = FP_W2.item()
    data["FN_W2"] = FN_W2.item()

    with open(file_path, "w") as f:
        json.dump(data, f, indent=2)

    # --- update best trackers ---
    avg_f1 = 0.5 * (f1_W1 + f1_W2)

    if f1_W1 > best_f1_W1:
        best_f1_W1 = f1_W1
        best_f1_W1_file = filename

    if f1_W2 > best_f1_W2:
        best_f1_W2 = f1_W2
        best_f1_W2_file = filename

    if avg_f1 > best_avg_f1:
        best_avg_f1 = avg_f1
        best_avg_f1_file = filename

# After the loop, print results
print("\n=== Best files ===")
print(f"Best f1_W1: {best_f1_W1:.4f} in file {best_f1_W1_file}")
print(f"Best f1_W2: {best_f1_W2:.4f} in file {best_f1_W2_file}")
print(f"Best average (f1_W1 + f1_W2)/2: {best_avg_f1:.4f} in file {best_avg_f1_file}")

Processing: result_d4_s3(new_reg)/result_d4_admg3_seed3_lambda0.1_lambda_corr0.01_lambda_nl1.0.json
Processing: result_d4_s3(new_reg)/result_d4_admg3_seed3_lambda0.01_lambda_corr0.1_lambda_nl8.0.json
Processing: result_d4_s3(new_reg)/result_d4_admg3_seed3_lambda0.1_lambda_corr0.01_lambda_nl12.0.json
Processing: result_d4_s3(new_reg)/result_d4_admg3_seed3_lambda0.1_lambda_corr0.1_lambda_nl12.0.json
Processing: result_d4_s3(new_reg)/result_d4_admg3_seed3_lambda0.1_lambda_corr0.01_lambda_nl4.0.json
Processing: result_d4_s3(new_reg)/result_d4_admg3_seed3_lambda0.01_lambda_corr0.1_lambda_nl4.0.json
Processing: result_d4_s3(new_reg)/result_d4_admg3_seed3_lambda0.01_lambda_corr0.01_lambda_nl12.0.json
Processing: result_d4_s3(new_reg)/result_d4_admg3_seed3_lambda0.01_lambda_corr0.001_lambda_nl1.0.json
Processing: result_d4_s3(new_reg)/result_d4_admg3_seed3_lambda0.001_lambda_corr0.1_lambda_nl8.0.json
Processing: result_d4_s3(new_reg)/result_d4_admg3_seed3_lambda0.001_lambda_corr0.1_lambda_nl1.

In [16]:
folder = "result_d4_s4(new_reg)"  
scores = []

for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    print("Processing:", file_path)
    with open(file_path, "r") as f:
        data = json.load(f)
        f1_W1 = data["f1_W1"]
        f1_W2 = data["f1_W2"]
        avg_f1 = 0.5 * (f1_W1 + f1_W2)

    scores.append((filename, f1_W1, f1_W2, avg_f1))

best_f1_W1 = max(s[1] for s in scores)
best_f1_W2 = max(s[2] for s in scores)
best_avg_f1 = max(s[3] for s in scores)

best_f1_W1_files = [f for f, s1, _, _ in scores if s1 == best_f1_W1]
best_f1_W2_files = [f for f, _, s2, _ in scores if s2 == best_f1_W2]
best_avg_f1_files = [f for f, _, _, a in scores if a == best_avg_f1]




Processing: result_d4_s4(new_reg)/result_d4_admg4_seed13_lambda0.01_lambda_corr0.1_lambda_nl12.0.json
Processing: result_d4_s4(new_reg)/result_d4_admg4_seed13_lambda0.001_lambda_corr0.1_lambda_nl4.0.json
Processing: result_d4_s4(new_reg)/result_d4_admg4_seed13_lambda0.1_lambda_corr0.001_lambda_nl8.0.json
Processing: result_d4_s4(new_reg)/result_d4_admg4_seed13_lambda0.01_lambda_corr0.001_lambda_nl8.0.json
Processing: result_d4_s4(new_reg)/result_d4_admg4_seed13_lambda0.1_lambda_corr0.001_lambda_nl1.0.json
Processing: result_d4_s4(new_reg)/result_d4_admg4_seed13_lambda0.1_lambda_corr0.1_lambda_nl1.0.json
Processing: result_d4_s4(new_reg)/result_d4_admg4_seed13_lambda0.001_lambda_corr0.001_lambda_nl8.0.json
Processing: result_d4_s4(new_reg)/result_d4_admg4_seed13_lambda0.01_lambda_corr0.01_lambda_nl12.0.json
Processing: result_d4_s4(new_reg)/result_d4_admg4_seed13_lambda0.001_lambda_corr0.001_lambda_nl4.0.json
Processing: result_d4_s4(new_reg)/result_d4_admg4_seed13_lambda0.1_lambda_corr

In [17]:
print(f"Best f1_W1: {best_f1_W1:.4f} in files:\n" +
      "\n".join(best_f1_W1_files))

Best f1_W1: 0.8000 in files:
result_d4_admg4_seed13_lambda0.1_lambda_corr0.001_lambda_nl8.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.001_lambda_nl1.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.1_lambda_nl1.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.1_lambda_nl4.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.01_lambda_nl4.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.001_lambda_nl4.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.001_lambda_nl12.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.01_lambda_nl8.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.01_lambda_nl12.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.1_lambda_nl12.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.01_lambda_nl1.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.1_lambda_nl8.0.json


In [18]:
print(f"Best f1_W2: {best_f1_W2:.4f} in files:\n" +
      "\n".join(best_f1_W2_files))

Best f1_W2: 0.6667 in files:
result_d4_admg4_seed13_lambda0.1_lambda_corr0.001_lambda_nl8.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.001_lambda_nl1.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.1_lambda_nl1.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.1_lambda_nl4.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.01_lambda_nl4.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.001_lambda_nl4.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.001_lambda_nl12.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.01_lambda_nl8.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.01_lambda_nl12.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.1_lambda_nl12.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.01_lambda_nl1.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.1_lambda_nl8.0.json


In [19]:
print(f"Best avg_f1: {best_avg_f1:.4f} in files:\n" +
      "\n".join(best_avg_f1_files))

Best avg_f1: 0.7333 in files:
result_d4_admg4_seed13_lambda0.1_lambda_corr0.001_lambda_nl8.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.001_lambda_nl1.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.1_lambda_nl1.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.1_lambda_nl4.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.01_lambda_nl4.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.001_lambda_nl4.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.001_lambda_nl12.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.01_lambda_nl8.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.01_lambda_nl12.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.1_lambda_nl12.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.01_lambda_nl1.0.json
result_d4_admg4_seed13_lambda0.1_lambda_corr0.1_lambda_nl8.0.json


In [23]:
folder = "result_d4_s4(new_reg)"  
filename = f"result_d4_admg4_seed13_lambda0.1_lambda_corr0.1_lambda_nl8.0.json" 
file_path = os.path.join(folder, filename)
with open(file_path, "r") as f:
        data = json.load(f)

for run in data["random_runs"]:
        if run["run_id"] == 0:
                chosen_run = run
                W_est = np.array(run["W_est_random"])
                Sigma_est = np.array(run["Sigma_est_random"])
                break
f1_W1 = data["f1_W1"]
f1_W2 = data["f1_W2"]
print("f1_W1: ", f1_W1)
print("f1_W2: ", f1_W2)
print("W_est: ", W_est)
print("Sigma_est: ", Sigma_est)

f1_W1:  0.8
f1_W2:  0.6667
W_est:  [[9.80039288e-11 6.23664007e-05 2.05851893e-04 1.08608430e-04]
 [6.33598465e-01 3.56612880e-11 3.90264849e-04 2.90933635e+00]
 [1.36246731e-01 6.50532301e-05 1.69359540e-11 1.55951067e+00]
 [7.18581070e-04 8.50695378e-05 1.86068836e-04 2.98457578e-10]]
Sigma_est:  [[ 5.14223049e-01 -8.80462703e-06 -2.40243870e-06  6.25228146e-01]
 [-8.80462703e-06  9.94097613e-01 -2.66511685e-02  3.60250644e-07]
 [-2.40243870e-06 -2.66511685e-02  1.02418292e+00 -2.27895064e-06]
 [ 6.25228146e-01  3.60250644e-07 -2.27895064e-06  1.01952580e+00]]


## Overall f1 

In [6]:
folder = "result_d4"  

W1_thres = 0.3
W2_thres = 0.2

# Track best files
best_f1_W1 = -1.0
best_f1_W1_file = None

best_f1_W2 = -1.0
best_f1_W2_file = None

best_avg_f1 = -1.0
best_avg_f1_file = None

for filename in os.listdir(folder):

    file_path = os.path.join(folder, filename)
    print("Processing:", file_path)

    with open(file_path, "r") as f:
        data = json.load(f)

    # --- truth ---
    W_truth = np.array(data["W_truth"])
    Sigma_truth = np.array(data["Sigma_truth"])  # assumes this key exists

    W_truth[np.abs(W_truth) < W1_thres] = 0
    Sigma_truth[np.abs(Sigma_truth) < W2_thres] = 0

    # --- pick run_id == 0 ---
    W_est = None
    Sigma_est = None
    chosen_run = None

    for run in data["random_runs"]:
        if run["run_id"] == 0:
            chosen_run = run
            W_est = np.array(run["W_est_random"])
            Sigma_est = np.array(run["Sigma_est_random"])
            break

    if W_est is None or Sigma_est is None:
        print(f"  Skipping {filename}: run_id == 0 not found.")
        continue

    # --- threshold estimates ---
    W_est[np.abs(W_est) < W1_thres] = 0
    Sigma_est[np.abs(Sigma_est) < W2_thres] = 0

    # --- F1 for W ---
    TP_W1, FP_W1, FN_W1, precision_W1, recall_W1, f1_W1 = f1_from_nonzero(W_truth, W_est)

    # --- F1 for Sigma (strictly lower-triangular) ---
    TP_W2, FP_W2, FN_W2, precision_W2, recall_W2, f1_W2 = f1_from_nonzero(
        np.tril(Sigma_truth, k=-1),
        np.tril(Sigma_est,   k=-1),
    )

    # --- save metrics into JSON ---
    data["f1_W1"] = round(f1_W1, 4)
    data["f1_W2"] = round(f1_W2, 4)
    # print("TP_W1", TP_W1)
    data["TP_W1"] = TP_W1.item()
    data["FP_W1"] = FP_W1.item()
    data["FN_W1"] = FN_W1.item()

    data["TP_W2"] = TP_W2.item()
    data["FP_W2"] = FP_W2.item()
    data["FN_W2"] = FN_W2.item()

    with open(file_path, "w") as f:
        json.dump(data, f, indent=2)

    # --- update best trackers ---
    avg_f1 = 0.5 * (f1_W1 + f1_W2)

    if f1_W1 > best_f1_W1:
        best_f1_W1 = f1_W1
        best_f1_W1_file = filename

    if f1_W2 > best_f1_W2:
        best_f1_W2 = f1_W2
        best_f1_W2_file = filename

    if avg_f1 > best_avg_f1:
        best_avg_f1 = avg_f1
        best_avg_f1_file = filename

# After the loop, print results
print("\n=== Best files ===")
print(f"Best f1_W1: {best_f1_W1:.4f} in file {best_f1_W1_file}")
print(f"Best f1_W2: {best_f1_W2:.4f} in file {best_f1_W2_file}")
print(f"Best average (f1_W1 + f1_W2)/2: {best_avg_f1:.4f} in file {best_avg_f1_file}")

Processing: result_d4/result_d4_admg2_seed1_lambda0.001_lambda_corr0.1_lambda_nl8.0.json
Processing: result_d4/result_d4_admg2_seed8_lambda0.001_lambda_corr0.1_lambda_nl8.0.json
Processing: result_d4/result_d4_admg2_seed0_lambda0.001_lambda_corr0.1_lambda_nl8.0.json
Processing: result_d4/result_d4_admg12_seed9_lambda0.001_lambda_corr0.1_lambda_nl8.0.json
Processing: result_d4/result_d4_admg2_seed4_lambda0.001_lambda_corr0.1_lambda_nl8.0.json
Processing: result_d4/result_d4_admg4_seed0_lambda0.001_lambda_corr0.1_lambda_nl8.0.json
Processing: result_d4/result_d4_admg12_seed8_lambda0.001_lambda_corr0.1_lambda_nl8.0.json
Processing: result_d4/result_d4_admg7_seed2_lambda0.001_lambda_corr0.1_lambda_nl8.0.json
Processing: result_d4/result_d4_admg3_seed9_lambda0.001_lambda_corr0.1_lambda_nl8.0.json
Processing: result_d4/result_d4_admg7_seed4_lambda0.001_lambda_corr0.1_lambda_nl8.0.json
Processing: result_d4/result_d4_admg4_seed2_lambda0.001_lambda_corr0.1_lambda_nl8.0.json
Processing: result_

In [7]:
TP_tot_W1 = 0
FP_tot_W1 = 0
FN_tot_W1 = 0

TP_tot_W2 = 0
FP_tot_W2 = 0
FN_tot_W2 = 0

for filename in os.listdir(folder):
    with open(os.path.join(folder, filename)) as f:
        data = json.load(f)

    TP_tot_W1 += data["TP_W1"]
    FP_tot_W1 += data["FP_W1"]
    FN_tot_W1 += data["FN_W1"]

    TP_tot_W2 += data["TP_W2"]
    FP_tot_W2 += data["FP_W2"]
    FN_tot_W2 += data["FN_W2"]

precision_W1 = TP_tot_W1 / (TP_tot_W1 + FP_tot_W1)
recall_W1    = TP_tot_W1 / (TP_tot_W1 + FN_tot_W1)
f1_W1        = 2 * precision_W1 * recall_W1 / (precision_W1+ recall_W1)

precision_W2 = TP_tot_W2 / (TP_tot_W2 + FP_tot_W2)
recall_W2    = TP_tot_W2 / (TP_tot_W2 + FN_tot_W2)
f1_W2        = 2 * precision_W2 * recall_W2 / (precision_W2+ recall_W2)

print("F1_W1:", f1_W1)
print("F1_W2:", f1_W2)


F1_W1: 0.8111111111111111
F1_W2: 0.8202247191011237
